In [15]:
!pip install kagglehub
!pip install numpy

You should consider upgrading via the 'D:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\project\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'D:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\project\Scripts\python.exe -m pip install --upgrade pip' command.


In [30]:
import shutil
import os
import kagglehub
import tqdm


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nrclex import NRCLex

In [66]:
nltk.download('punkt')
nltk.download('stopwords') 
nltk.download('averaged_perceptron_tagger')  

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [67]:
!python -m textblob.download_corpora

Finished.


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\jaeseokk\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is alread

### Kaggle Dataset Load

In [8]:
# Download latest version
path = kagglehub.dataset_download("deepshah16/song-lyrics-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\jaeseokk\.cache\kagglehub\datasets\deepshah16\song-lyrics-dataset\versions\5


In [3]:
current_directory = os.getcwd()
print("Current Path:", current_directory)

destination_path = os.path.join(current_directory, "song-lyrics-dataset")
shutil.copytree(path, destination_path, dirs_exist_ok=True)

print("Dataset directory has been copied to:", destination_path)

Current Path: d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project


NameError: name 'path' is not defined

Lyrics datasets have been saved in `song-lyrics-dataset/csv`.

### Lyrics Dataset Handling

In [3]:
current_directory = os.getcwd()

csv_path = os.path.join(current_directory, "song-lyrics-dataset", "csv")

artists = os.listdir(csv_path)
for artist in artists:
    lyrics_data = os.path.join(csv_path,artist)
    print(lyrics_data)
    df1 = pd.read_csv(lyrics_data)

# 다 합치는 코드 작성해야함
    
df1.head()


d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\ArianaGrande.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Beyonce.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\BillieEilish.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\BTS.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\CardiB.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\CharliePuth.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\ColdPlay.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Drake.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\DuaLipa.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\EdSheeran.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Project\song-lyrics-dataset\csv\Eminem.csv
d:\융시공\3학년 2학기\Ajou_SocialNetworkAnalysis\Pro

,Unnamed: 0,Artist,Title,Album,Year,Date,Lyric
0,0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...
1,1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...
2,2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...
3,3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...
4,4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...


### Lyrics Analysis
##### 1. Knowledge-based Method 

In [5]:
# VADER 감성 분석기 초기화
analyzer = SentimentIntensityAnalyzer()

In [6]:
def analyze_sentiment(text):
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']  # 전체적인 감정 점수 (-1~1)
    # 긍정, 부정, 중립으로 분류
    if compound >= 0.05:
        return "Positive"
    elif compound <= -0.05:
        return "Negative"
    else:
        return "Neutral"

In [16]:
def analyze_emotions(text):
    emotions = NRCLex(text).raw_emotion_scores
    # 감정이 없는 경우 0으로 채움
    for emotion in ['joy', 'sadness', 'anger', 'fear', 'anticipation', 'trust', 'surprise', 'disgust']:
        if emotion not in emotions:
            emotions[emotion] = 0
    return emotions


In [17]:
def dominant_emotion(emotion_scores):
    return max(emotion_scores, key=emotion_scores.get)

In [ ]:
df1['Lyric'] = df1['Lyric'].apply(str)
df1['Sentiment'] = df1['Lyric'].apply(analyze_sentiment)
df1['Compound Score'] = df1['Lyric'].apply(lambda x: analyzer.polarity_scores(x)['compound'])


In [ ]:
df1['Emotion Scores'] = df1['Lyric'].apply(analyze_emotions)
df1['Dominant Emotion'] = df1['Emotion Scores'].apply(dominant_emotion)

MissingCorpusError: 
Looks like you are missing some required data for this feature.

To download the necessary data, simply run

    python -m textblob.download_corpora

or use the NLTK downloader to download the missing data: http://nltk.org/data.html
If this doesn't fix the problem, file an issue at https://github.com/sloria/TextBlob/issues.


In [ ]:
df1.head()

TypeError: cannot do positional indexing on RangeIndex with these indexers [C:\Users\jaeseokk\AppData\Roaming\nltk_data\tokenizers\punkt] of type str

##### 2. Statistical Method

##### 3. Deep Learning Method

In [6]:
artists

['ArianaGrande.csv',
 'Beyonce.csv',
 'BillieEilish.csv',
 'BTS.csv',
 'CardiB.csv',
 'CharliePuth.csv',
 'ColdPlay.csv',
 'Drake.csv',
 'DuaLipa.csv',
 'EdSheeran.csv',
 'Eminem.csv',
 'JustinBieber.csv',
 'KatyPerry.csv',
 'Khalid.csv',
 'LadyGaga.csv',
 'Maroon5.csv',
 'NickiMinaj.csv',
 'PostMalone.csv',
 'Rihanna.csv',
 'SelenaGomez.csv',
 'TaylorSwift.csv']